# `provabgs` SED modeling to infer $M_*$ using photometry
We use `provabgs` to infer $M_*$ from forward modeled photometry

In [1]:
import os
import numpy as np
from astropy import table as aTable

In [2]:
from provabgs import util as UT
from provabgs import infer as Infer
from provabgs import models as Models

import speclite.filters

/Users/chahah/projects/provabgs/src/provabgs/models.py:29: UserWarning: import error with pytorch; cannot use msurv emulator
  warnings.warn('import error with pytorch; cannot use msurv emulator')


In [3]:
# -- plotting -- 
import corner as DFM
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

## read test photometry
We'll just be using the magnitudes

In [4]:
subhalo_test = aTable.Table.read('/Users/chahah/data/frb_halos/subhalos.central.snapshot91.test.csv')

I forgot to add magnitude uncertainties, so I'm doing it here. 

In [5]:
morphs = aTable.Table.read('/Users/chahah/data/frb_halos/MorphSersic_HSC_combined.csv')

In [6]:
cols = ['snapshot', 'subhalo_id', 'version'] + ['%s_Sersic_dmag_m' % b for b in ['g', 'r', 'i', 'z', 'y']] + ['%s_Sersic_dmag_p' % b for b in ['g', 'r', 'i', 'z', 'y']]
subhalo_test = aTable.join(subhalo_test, morphs[cols], keys=['snapshot', 'subhalo_id', 'version'])

In [7]:
subhalo_test[:5]

g_Sersic_Reff,g_Sersic_mag,g_Sersic_dmag_m_1,g_Sersic_dmag_p_1,g_CAS_C,g_CAS_A,snapshot,subhalo_id,version,i_Sersic_Reff,i_Sersic_mag,i_Sersic_dmag_m_1,i_Sersic_dmag_p_1,i_CAS_C,i_CAS_A,r_Sersic_Reff,r_Sersic_mag,r_Sersic_dmag_m_1,r_Sersic_dmag_p_1,r_CAS_C,r_CAS_A,y_Sersic_Reff,y_Sersic_mag,y_Sersic_dmag_m_1,y_Sersic_dmag_p_1,y_CAS_C,y_CAS_A,z_Sersic_Reff,z_Sersic_mag,z_Sersic_dmag_m_1,z_Sersic_dmag_p_1,z_CAS_C,z_CAS_A,SubhaloMassType_stars,SubhaloMassType_dm,g_Sersic_dmag_m_2,r_Sersic_dmag_m_2,i_Sersic_dmag_m_2,z_Sersic_dmag_m_2,y_Sersic_dmag_m_2,g_Sersic_dmag_p_2,r_Sersic_dmag_p_2,i_Sersic_dmag_p_2,z_Sersic_dmag_p_2,y_Sersic_dmag_p_2
float64,float64,float64,float64,float64,float64,int64,int64,str2,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
9.43492,14.7362,-0.0228001,0.0113216,5.675,0.151536,91,188893,v0,7.58215,13.4072,-0.0368903,0.0111382,5.65709,0.134309,8.32708,13.8716,-0.00479359,0.00371598,5.76436,0.132757,8.17655,12.9869,-0.00254906,0.00518571,5.44145,0.0832174,7.40989,13.1486,-0.000101163,0.000100114,5.65709,0.1109,12.0927,13.4813,-0.0228001,-0.00479359,-0.0368903,-0.000101163,-0.00254906,0.0113216,0.00371598,0.0111382,0.000100114,0.00518571
6.49051,14.9074,-0.00123298,0.00651073,5.755,0.182424,91,188893,v1,7.04467,13.4726,-0.000148445,6.24312e-05,5.64614,0.163196,6.05461,14.0322,-0.0237968,0.0188774,5.42976,0.140707,6.37337,13.114,-0.0109762,0.00327845,5.42702,0.12831,7.33593,13.179,-0.00392127,0.0288633,5.53682,0.133344,12.0927,13.4813,-0.00123298,-0.0237968,-0.000148445,-0.00392127,-0.0109762,0.00651073,0.0188774,6.24312e-05,0.0288633,0.00327845
17.5121,15.001,-0.000498276,0.000607144,3.97173,0.329825,91,188893,v2,19.1861,13.4974,-0.0159896,0.006454,4.48426,0.288616,17.2919,14.1008,-0.0155399,0.00920959,4.18496,0.297089,15.1372,13.1519,-0.103126,0.0536822,4.70154,0.233608,15.1383,13.3053,-0.0481313,0.0152788,4.58445,0.254776,12.0927,13.4813,-0.000498276,-0.0155399,-0.0159896,-0.0481313,-0.103126,0.000607144,0.00920959,0.006454,0.0152788,0.0536822
7.34427,15.0605,-0.000364375,0.00395798,5.97144,0.359144,91,188893,v3,5.22868,13.7271,-0.000163293,0.000979249,5.64614,0.271607,7.14591,14.1488,-0.0176715,0.029636,5.755,0.295808,6.27765,13.2321,-0.000139421,0.000166413,5.65596,0.212494,5.70168,13.4,-0.000151222,0.000169109,5.42702,0.227725,12.0927,13.4813,-0.000364375,-0.0176715,-0.000163293,-0.000151222,-0.000139421,0.00395798,0.029636,0.000979249,0.000169109,0.000166413
0.747813,16.6905,-0.000183699,0.000163322,4.83485,0.0308228,91,208563,v0,0.632306,15.3027,-4.14848e-05,3.71917e-05,4.85612,0.0242648,0.646457,15.82,-6.01077e-05,6.41379e-05,4.97316,0.0179406,0.594059,14.92,-9.84968e-05,0.000108269,4.15432,0.0150189,0.622927,15.0025,-6.39485e-05,6.19838e-05,4.49907,0.0281371,11.7227,13.1865,-0.000183699,-6.01077e-05,-4.14848e-05,-6.39485e-05,-9.84968e-05,0.000163322,6.41379e-05,3.71917e-05,6.19838e-05,0.000108269


## set up `provabgs`

In [8]:
# read in HSC photometry filters
dat_filter = '/opt/anaconda3/envs/gqp/lib/python3.7/site-packages/speclite/data/filters/'
hsc = speclite.filters.FilterSequence([speclite.filters.load_filter(
    os.path.join(dat_filter, 'hsc2017-%s.ecsv') % b) for b in ['g', 'r', 'i', 'z', 'y']])

In [9]:
# declare prior
prior = Infer.load_priors([
    Infer.UniformPrior(7., 13.0, label='sed'),
    Infer.FlatDirichletPrior(4, label='sed'),   # flat dirichilet priors
    Infer.UniformPrior(0., 1., label='sed'), # burst fraction
    Infer.UniformPrior(1e-2, 13.27, label='sed'), # tburst
    Infer.LogUniformPrior(4.5e-5, 1.5e-2, label='sed'), # log uniform priors on ZH coeff
    Infer.LogUniformPrior(4.5e-5, 1.5e-2, label='sed'), # log uniform priors on ZH coeff
    Infer.UniformPrior(0., 3., label='sed'),        # uniform priors on dust1
    Infer.UniformPrior(0., 3., label='sed'),        # uniform priors on dust2
    Infer.UniformPrior(-2., 1., label='sed')    # uniform priors on dust_index
])

In [10]:
# declare SPS model
m_nmf = Models.NMF(burst=True, emulator=True)

input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index


In [11]:
# MCMC object
mcmc = Infer.specphotoMCMC(model=m_nmf, prior=prior)

In [12]:
mcmc.photometric_filters = hsc

In [ ]:

chain = mcmc.run(
        wave_obs=w_obs, # observed wavelength
        flux_obs=f_obs, # observed flux of spectrum
        flux_ivar_obs=np.ones(len(f_obs)), # no noise in this example
        zred=0.1,       # redshift
        vdisp=0.,       # velocity dispersion (set to 0 for simplicity)
        sampler='zeus', # zeus ensemble slice sample
        nwalkers=30,    # number of MCMC walkers
        burnin=500,     # burn in iterations 
        opt_maxiter=2000, # maximum number of iterations for initial optimizer
        niter=3000,     # number of iterations after burn in
        progress=True)  # show progress bar